In [47]:
import numpy as np
import pandas as pd

In [48]:
random_state = 100
cv_folds = 5

## Data Loading

In [49]:
df_features = pd.read_csv("compiled_features_dataset.csv")
df_features.head()

,Unnamed: 0,path,source_w,source_h,face_index,x1,y1,x2,y2,used_mask,...,e_face_yf,e_bg_yn,e_bg_mp,e_bg_yf,e_bbox_yn,e_bbox_mp,e_bbox_yf,e_lbbox_yn,e_lbbox_mp,e_lbbox_yf
0,0,C:\Users\jayba\Documents\GitHub\THS-ST1\images...,683,1024,75,339,207,346,215,True,...,0.190,0.0,0.0,3.1,0.0,0.0,0.190,0.00,0.0,0.115
1,1,C:\Users\jayba\Documents\GitHub\THS-ST1\images...,899,1024,41,200,95,205,102,True,...,0.845,0.0,0.0,3.1,0.0,0.0,0.845,0.00,0.0,3.100
2,2,C:\Users\jayba\Documents\GitHub\THS-ST1\images...,768,1024,23,116,214,120,220,True,...,0.540,0.0,0.0,3.1,0.0,0.0,0.540,0.00,0.0,0.350
3,3,C:\Users\jayba\Documents\GitHub\THS-ST1\images...,768,1024,21,149,182,158,191,True,...,0.205,3.1,0.0,3.1,3.1,0.0,0.205,0.53,0.0,0.200
4,4,C:\Users\jayba\Documents\GitHub\THS-ST1\images...,768,1024,1,380,252,383,257,True,...,0.420,0.0,0.0,3.1,0.0,0.0,0.420,0.00,0.0,0.330


In [50]:
print(df_features.columns)

Index(['Unnamed: 0', 'path', 'source_w', 'source_h', 'face_index', 'x1', 'y1',
       'x2', 'y2', 'used_mask',
       ...
       'e_face_yf', 'e_bg_yn', 'e_bg_mp', 'e_bg_yf', 'e_bbox_yn', 'e_bbox_mp',
       'e_bbox_yf', 'e_lbbox_yn', 'e_lbbox_mp', 'e_lbbox_yf'],
      dtype='object', length=1050)


In [51]:
df_features = df_features.drop(columns = ['Unnamed: 0', 'path', 'source_w', 'source_h', 'face_index'])
df_features = df_features.loc[df_features["e_bbox_yf"] < 3, :]
print(df_features.columns)

Index(['x1', 'y1', 'x2', 'y2', 'used_mask', 'R_BIN_mask_0', 'R_BIN_mask_1',
       'R_BIN_mask_2', 'R_BIN_mask_3', 'R_BIN_mask_4',
       ...
       'e_face_yf', 'e_bg_yn', 'e_bg_mp', 'e_bg_yf', 'e_bbox_yn', 'e_bbox_mp',
       'e_bbox_yf', 'e_lbbox_yn', 'e_lbbox_mp', 'e_lbbox_yf'],
      dtype='object', length=1045)


Change string types to numeric types

In [52]:
from sklearn.preprocessing import LabelEncoder

In [53]:
encoder = LabelEncoder()
categorical_columns = df_features.select_dtypes(include=[bool, object]).columns
encoded_columns = df_features[categorical_columns].apply(encoder.fit_transform)
encoded_columns

,used_mask
0,1
1,1
2,1
3,1
4,1
...,...
5141,1
5142,1
5143,1
5144,1


In [54]:
df_encoded_features = df_features.copy()
df_encoded_features[categorical_columns] = encoded_columns
df_encoded_features

,x1,y1,x2,y2,used_mask,R_BIN_mask_0,R_BIN_mask_1,R_BIN_mask_2,R_BIN_mask_3,R_BIN_mask_4,...,e_face_yf,e_bg_yn,e_bg_mp,e_bg_yf,e_bbox_yn,e_bbox_mp,e_bbox_yf,e_lbbox_yn,e_lbbox_mp,e_lbbox_yf
0,339,207,346,215,1,0.00000,0.0,0.000000,0.000000,0.000000,...,0.190,0.0,0.0,3.1,0.000,0.000,0.190,0.00,0.00,0.115
1,200,95,205,102,1,0.00000,0.0,0.200000,0.000000,0.000000,...,0.845,0.0,0.0,3.1,0.000,0.000,0.845,0.00,0.00,3.100
2,116,214,120,220,1,0.00000,0.0,0.000000,0.000000,0.000000,...,0.540,0.0,0.0,3.1,0.000,0.000,0.540,0.00,0.00,0.350
3,149,182,158,191,1,0.00000,0.0,0.000000,0.000000,0.000000,...,0.205,3.1,0.0,3.1,3.100,0.000,0.205,0.53,0.00,0.200
4,380,252,383,257,1,0.00000,0.0,0.000000,0.000000,0.000000,...,0.420,0.0,0.0,3.1,0.000,0.000,0.420,0.00,0.00,0.330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5141,131,101,300,376,1,0.00000,0.0,0.000000,0.242604,0.177515,...,0.155,3.5,3.5,3.5,0.455,0.180,0.140,3.10,0.21,0.120
5142,139,120,291,340,1,0.00000,0.0,0.000000,0.000000,0.000000,...,0.455,3.5,3.5,3.5,0.175,0.240,0.460,3.10,0.24,0.180
5143,122,65,301,309,1,0.22905,0.0,0.000000,0.000000,0.050279,...,0.265,3.5,3.5,3.5,0.175,0.165,0.220,3.10,0.15,0.130
5144,122,97,272,307,1,0.00000,0.0,0.106667,0.153333,0.273333,...,0.225,3.5,3.5,3.5,0.175,0.205,0.205,3.10,0.21,0.175


Split into X and Y

In [55]:
features = ["R_BIN_bbox_" + str(i) for i in range(26)]
features += ["G_BIN_bbox_" + str(i) for i in range(26)]
features += ["B_BIN_bbox_" + str(i) for i in range(26)]
features += ["LBP_BIN_bbox_" + str(i) for i in range(26)]
features += ["SOBELX_BIN_bbox_" + str(i) for i in range(26)]
features += ["SOBELY_BIN_bbox_" + str(i) for i in range(26)]
features += ["SOBEL_BIN_mask_" + str(i) for i in range(26)]
X_features =  df_encoded_features.loc[:,  features]
y_features = df_encoded_features.loc[:, "e_bbox_yf"].values

In [56]:
from sklearn.model_selection import train_test_split

In [57]:
# write code here
X_train, X_test, y_train, y_test = train_test_split(X_features, y_features, test_size = 0.2, random_state=random_state)
X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size = 0.25, random_state=random_state)

print("Split shapes")
print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_val: ", X_val.shape)
print("y_val: ", y_val.shape)
print("X_test: ", X_test.shape)
print("y_test: ", y_test.shape)

Split shapes
X_train:  (3073, 182)
y_train:  (3073,)
X_val:  (1025, 182)
y_val:  (1025,)
X_test:  (1025, 182)
y_test:  (1025,)


In [58]:
n_samples = df_features.shape[0]
n_samples

5123

## Base Models

### Model Evaluation Functions

In [66]:
def rmse(predictions, actual):
    return np.sqrt(np.mean(np.square(predictions - actual)))

In [67]:
def mae(predictions, actual):
    return np.mean(np.abs(predictions - actual))

### Random Forest

Train RF model

In [17]:
from sklearn.ensemble import RandomForestRegressor

In [18]:
rfr = RandomForestRegressor(random_state = random_state)
rfr.fit(X_train, y_train)

RandomForestRegressor(random_state=100)

In [20]:
rfr.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 100,
 'verbose': 0,
 'warm_start': False}

In [37]:
rfr_pred = rfr.predict(X_val)
rfr_pred

array([0.17145, 0.16725, 0.11775, ..., 0.15705, 0.202  , 0.15525])

In [38]:
print(rmse(rfr_pred, y_val))
print(min(y_val), max(y_val))

0.068966769711146
0.0049999999999999 0.8450000000000004


In [39]:
print("STD:", np.std(y_val))
print("Mean:", np.mean(y_val))

STD: 0.0835114491646648
Mean: 0.19350731707317065


### Support Vector Machine

In [44]:
from sklearn.svm import SVR
svr = SVR()
svr.fit(X_train, y_train)

SVR()

In [ ]:
svr_pred = mlp.predict(X_val)
svr_pred

In [ ]:
rmse(svr_pred, y_test)

### Multilayer Perceptrons



In [18]:
from sklearn.neural_network import MLPRegressor

In [32]:
mpr = MLPRegressor(random_state = random_state)
mpr.fit(X_train, y_train)

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [33]:
mpr_pred = mlp.predict(X_val)
mpr_pred

0.9013333333333333


In [ ]:
rmse(mpr_pred, y_test)

## Feature selection and Hyperparameter tuning



In [69]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.feature_selection import RFE

### Random Forest

**Grid Search**

```
param_space = [
    {
        "n_estimators": list(range(100, 351, 50)),
        "criterion": ["squared_error", "absolute_error", "friedman_mse", "poisson"],
        "max_depth": [None, 1, 10, 30, 80, 150],
        "min_samples_split": [2, 4, 8, 16, 32],
        "min_samples_leaf": [1, 5, 10, 20],
        "max_features": [None, "sqrt", "log2", 0.25, 0.5, 0.75],
        "max_leaf_nodes": [None, 50, 100, 300],
        "min_impurity_decrease": [0.0, 0.5],
        "bootstrap": [False, True],
        "ccp_alpha": [0.0, 0.5, 1.0],
    }
]
```

```
rfr = RandomForestRegressor()
gs_rfr = GridSearchCV(rfr, param_space, n_jobs=-1, cv=cv_folds)
gs_rfr.fit(X_train, y_train)
```

```
print('Best parameters found:\n', gs_svr.best_params_)
y_pred = gs_rfr.predict(X_val)
print("Best accuracy: ", rmse(y_val, y_pred))
print(classification_report(y_val, y_pred))
```

**Bayesian Optimization**

In [70]:
search_space = {
    "n_estimators": Integer(100, 350),
    "criterion": Categorical(["squared_error", "absolute_error", "friedman_mse", "poisson"]),
    "max_depth": Integer(1, 300),
    "min_samples_split": Integer(2, 32),
    "min_samples_leaf": Integer(1, 20),
    "max_features": Categorical([None, "sqrt", "log2", 0.25, 0.5, 0.75]),
    "max_leaf_nodes": Integer(50, 300),
    "min_impurity_decrease": Real(0.0, 2.0),
    "bootstrap": Categorical([False, True]),
    "ccp_alpha": Real(0.0, 2.0),
}

In [71]:
rfr = RandomForestRegressor()
bo_rfr = BayesSearchCV(rfr, search_space, n_iter=20, n_jobs=-1, cv=cv_folds, random_state=random_state)
bo_rfr.fit(X_train, y_train)

BayesSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=20, n_jobs=-1,
              random_state=100,
              search_spaces={'bootstrap': Categorical(categories=(False, True), prior=None),
                             'ccp_alpha': Real(low=0.0, high=2.0, prior='uniform', transform='normalize'),
                             'criterion': Categorical(categories=('squared_error', 'absolute_error', 'friedman_mse', 'poisson'), prior=None),
                             'max_...
                             'max_leaf_nodes': Integer(low=50, high=300, prior='uniform', transform='normalize'),
                             'min_impurity_decrease': Real(low=0.0, high=2.0, prior='uniform', transform='normalize'),
                             'min_samples_leaf': Integer(low=1, high=20, prior='uniform', transform='normalize'),
                             'min_samples_split': Integer(low=2, high=32, prior='uniform', transform='normalize'),
                             'n_estimators': Integer(low=100, high=350, prior='uniform', transform='normalize')})

In [72]:
print('Best parameters found:\n', bo_rfr.best_params_)
y_pred = bo_rfr.predict(X_val)
print("RMSE: ", rmse(y_val, y_pred))
print("MAE: ", mae(y_val, y_pred))

Best parameters found:
 OrderedDict([('bootstrap', True), ('ccp_alpha', 1.2863366672065861), ('criterion', 'friedman_mse'), ('max_depth', 186), ('max_features', 0.5), ('max_leaf_nodes', 268), ('min_impurity_decrease', 1.324946323219672), ('min_samples_leaf', 16), ('min_samples_split', 31), ('n_estimators', 188)])
RMSE:  0.08351384068981149
MAE:  0.05998037552238212


### Feature Selection

In [80]:
rfr = RandomForestRegressor(**bo_rfr.best_params_)
selector = RFE(rfr, step=1)
selector = selector.fit(X_train, y_train)
selector.ranking_

array([92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76,
       75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59,
       58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42,
       41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25,
       24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10,  9,  8,
        7,  6,  5,  4,  3,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1])

In [88]:
rfr = RandomForestRegressor()
bo_rfr = BayesSearchCV(rfr, search_space, n_iter=50, n_jobs=-1, cv=cv_folds, random_state=random_state)
bo_rfr.fit(X_train.loc[:, selector.support_], y_train)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- B_BIN_bbox_0
- B_BIN_bbox_1
- B_BIN_bbox_10
- B_BIN_bbox_11
- B_BIN_bbox_12
- ...


In [89]:
y_pred = bo_rfr.predict(X_val.loc[:, selector.support_])

print('Best features found:\n', X_train.columns[selector.support_])
print("RMSE: ", rmse(y_val, y_pred))
print("MAE: ", mae(y_val, y_pred))

Best features found:
 Index(['LBP_BIN_bbox_13', 'LBP_BIN_bbox_14', 'LBP_BIN_bbox_15',
       'LBP_BIN_bbox_16', 'LBP_BIN_bbox_17', 'LBP_BIN_bbox_18',
       'LBP_BIN_bbox_19', 'LBP_BIN_bbox_20', 'LBP_BIN_bbox_21',
       'LBP_BIN_bbox_22', 'LBP_BIN_bbox_23', 'LBP_BIN_bbox_24',
       'LBP_BIN_bbox_25', 'SOBELX_BIN_bbox_0', 'SOBELX_BIN_bbox_1',
       'SOBELX_BIN_bbox_2', 'SOBELX_BIN_bbox_3', 'SOBELX_BIN_bbox_4',
       'SOBELX_BIN_bbox_5', 'SOBELX_BIN_bbox_6', 'SOBELX_BIN_bbox_7',
       'SOBELX_BIN_bbox_8', 'SOBELX_BIN_bbox_9', 'SOBELX_BIN_bbox_10',
       'SOBELX_BIN_bbox_11', 'SOBELX_BIN_bbox_12', 'SOBELX_BIN_bbox_13',
       'SOBELX_BIN_bbox_14', 'SOBELX_BIN_bbox_15', 'SOBELX_BIN_bbox_16',
       'SOBELX_BIN_bbox_17', 'SOBELX_BIN_bbox_18', 'SOBELX_BIN_bbox_19',
       'SOBELX_BIN_bbox_20', 'SOBELX_BIN_bbox_21', 'SOBELX_BIN_bbox_22',
       'SOBELX_BIN_bbox_23', 'SOBELX_BIN_bbox_24', 'SOBELX_BIN_bbox_25',
       'SOBELY_BIN_bbox_0', 'SOBELY_BIN_bbox_1', 'SOBELY_BIN_bbox_2',
       '

### Support Vector Machine

**Grid Search**

```
param_space = [
    {
        "kernel": "poly",
        "degree": [3, 5, 10, 15, 20],
        "gamma": ["auto", "scale"],
        "coef0": [0.0, 2.5, 5.0],
        "tol": [0.0001, 0.001, 0.01, 0.1],
        "C": [0.0001, 0.01, 1.0, 100.0, 1000.0],
        "epsilon": [0.05, 0.1, .5],
        "shrinking": [False, True],
        "max_iter": [-1, 100, 500, 1000],
    },
    {
        "kernel": "rbf",
        "gamma": ["auto", "scale"],
        "tol": [0.0001, 0.001, 0.01, 0.1],
        "C": [0.0001, 0.01, 1.0, 100.0, 1000.0],
        "epsilon": [0.05, 0.1, .5],
        "shrinking": [False, True],
        "max_iter": [-1, 100, 500, 1000],
    },
    {
        "kernel": "sigmoid",
        "gamma": ["auto", "scale"],
        "coef0": [0.0, 2.5, 5.0],
        "tol": [0.0001, 0.001, 0.01, 0.1],
        "C": [0.0001, 0.01, 1.0, 100.0, 1000.0],
        "epsilon": [0.05, 0.1, .5],
        "shrinking": [False, True],
        "max_iter": [-1, 100, 500, 1000],
    },
    {
        "kernel": ["linear"],
        "tol": [0.0001, 0.001, 0.01, 0.1],
        "C": [0.0001, 0.01, 1.0, 100.0, 1000.0],
        "epsilon": [0.05, 0.1, .5],
        "shrinking": [False, True],
        "max_iter": [-1, 100, 500, 1000],
    }
]
```

```
svr = SVR()
gs_svr = GridSearchCV(svr, param_space, n_jobs=-1, cv=cv_folds)
gs_svr.fit(X_train, y_train)
```

```
print('Best parameters found:\n', gs_svr.best_params_)
y_pred = gs_svr.predict(X_val)
print("Best accuracy: ", rmse(y_val, y_pred))
print(classification_report(y_val, y_pred))
```

**Bayesian Optimization**

In [46]:
search_space = {
    "kernel": Categorical(["linear", "poly", "rbf", "sigmoid"]),
    "degree": Integer(3, 20),
    "gamma": Categorical(["auto", "scale"]),
    "coef0": Real(0.0, 5.0),
    "tol": Real(0.0001, 0.1),
    "C": Real(0.0001, 1000.0),
    "epsilon": Real(0.05, .5),
    "shrinking": Categorical([False, True]),
    "max_iter": Integer(100, 5000),
}

In [ ]:
svr = SVR()
bo_svr = BayesSearchCV(svr, search_space, n_iter=20, n_jobs=-1, cv=cv_folds, random_state=random_state)
bo_svr.fit(X_train, y_train)

In [ ]:
print('Best parameters found:\n', bo_svr.best_params_)
y_pred = bo_svr.predict(X_val)
print("RMSE: ", rmse(y_val, y_pred))
print("MAE: ", mae(y_val, y_pred))

### Feature Selection

In [31]:
svr = SVR(**bo_svr.best_params_)
selector = RFE(svr, step=1)
selector = selector.fit(X_train, y_train)
selector.ranking_

array([17,  1,  1,  1,  1,  8,  4,  1,  1,  1,  1,  1,  1, 12,  1,  6, 20,
       10, 22, 19,  1, 11,  1,  1, 25, 28,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  1, 18,  1,  3,
       21,  1,  1,  1,  1,  1,  1,  1,  1,  5,  1,  1,  1,  1,  1,  1,  1,
        9,  1,  1, 15, 14,  1,  1,  1, 13, 24,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  7,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1, 26, 30, 60, 53, 42, 57, 71, 68, 40, 43, 44, 46,
       48, 45, 54, 50, 58, 62, 63, 64, 67, 72, 66,  1,  1,  1, 23, 31, 74,
       56, 65, 41, 70, 69, 61, 73, 52, 59, 49, 55, 51, 38, 47, 36, 39, 33,
       34, 35, 37,  1,  1,  1,  1, 16, 27, 29, 32, 75, 76, 77, 78, 79, 80,
       81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92])

In [ ]:
svr = SVR()
bo_svr = BayesSearchCV(svr, search_space, n_iter=50, n_jobs=-1, cv=cv_folds, random_state=random_state)
bo_svr.fit(X_train.loc[:, selector.support_], y_train)

In [89]:
y_pred = bo_svr.predict(X_val.loc[:, selector.support_])

print('Best features found:\n', X_train.columns[selector.support_])
print("RMSE: ", rmse(y_val, y_pred))
print("MAE: ", mae(y_val, y_pred))

Best features found:
 Index(['LBP_BIN_bbox_13', 'LBP_BIN_bbox_14', 'LBP_BIN_bbox_15',
       'LBP_BIN_bbox_16', 'LBP_BIN_bbox_17', 'LBP_BIN_bbox_18',
       'LBP_BIN_bbox_19', 'LBP_BIN_bbox_20', 'LBP_BIN_bbox_21',
       'LBP_BIN_bbox_22', 'LBP_BIN_bbox_23', 'LBP_BIN_bbox_24',
       'LBP_BIN_bbox_25', 'SOBELX_BIN_bbox_0', 'SOBELX_BIN_bbox_1',
       'SOBELX_BIN_bbox_2', 'SOBELX_BIN_bbox_3', 'SOBELX_BIN_bbox_4',
       'SOBELX_BIN_bbox_5', 'SOBELX_BIN_bbox_6', 'SOBELX_BIN_bbox_7',
       'SOBELX_BIN_bbox_8', 'SOBELX_BIN_bbox_9', 'SOBELX_BIN_bbox_10',
       'SOBELX_BIN_bbox_11', 'SOBELX_BIN_bbox_12', 'SOBELX_BIN_bbox_13',
       'SOBELX_BIN_bbox_14', 'SOBELX_BIN_bbox_15', 'SOBELX_BIN_bbox_16',
       'SOBELX_BIN_bbox_17', 'SOBELX_BIN_bbox_18', 'SOBELX_BIN_bbox_19',
       'SOBELX_BIN_bbox_20', 'SOBELX_BIN_bbox_21', 'SOBELX_BIN_bbox_22',
       'SOBELX_BIN_bbox_23', 'SOBELX_BIN_bbox_24', 'SOBELX_BIN_bbox_25',
       'SOBELY_BIN_bbox_0', 'SOBELY_BIN_bbox_1', 'SOBELY_BIN_bbox_2',
       '

### Multilayer Perceptron

**Grid Search**

```
param_space = [
    {
        "solver": "lbfgs",
        "hidden_layer_sizes": [(100,), (50, 50,), (50, 25, 25,)], # pick better ones
        "activation": ["identity", "logistic", "tanh", "relu"],
        "alpha": [0.00001, 0.0001, 0.001],
        "max_iter": [200, 500, 1000],
        "random_state": random_state,
        "tol": [0.0001, 0.001, 0.01, 0.1],
        "max_fun": [10000, 15000],
    },
    {
        "solver": "adam",
        "hidden_layer_sizes": [(100,), (50, 50,), (50, 25, 25,)], # pick better ones
        "activation": ["identity", "logistic", "tanh", "relu"],
        "alpha": [0.00001, 0.0001, 0.001],
        "max_iter": [200, 500, 1000],
        "random_state": random_state,
        "tol": [0.0001, 0.001, 0.01, 0.1],
        "batch_size": ['auto', n_samples // 5, n_samples // 10],
        "learning_rate_init": [0.0005, 0.001, 0.005],
        "shuffle": [False, True],
        "early_stopping": [False, True],
        "validation_fraction": [0.1, 0.15],
        "n_iter_no_change": [10, 15],
        # not sure abt these values, should we be changing these?
        "beta_1": [0.75, 0.9],
        "beta_2": [0.85, 0.999],
        "epsilon": [1e-07, 1e-08],
    },
    {
        "solver": "sgd",
        "hidden_layer_sizes": [(100,), (50, 50,), (50, 25, 25,)], # pick better ones
        "activation": ["identity", "logistic", "tanh", "relu"],
        "alpha": [0.00001, 0.0001, 0.001],
        "max_iter": [200, 500, 1000],
        "random_state": random_state,
        "tol": [0.0001, 0.001, 0.01, 0.1],
        "batch_size": ['auto', n_samples // 5, n_samples // 10],
        "learning_rate_init": [0.0005, 0.001, 0.005],
        "shuffle": [False, True],
        "early_stopping": [False, True],
        "validation_fraction": [0.1, 0.15],
        "n_iter_no_change": [10, 15],
        "learning_rate": ["constant", "invscaling", "adaptive"],
        "momentum": [0.75, 0.9],
        "nesterovs_momentum": [False, True],
    }
]
```

```
mpr = MLPRegressor()
gs_mpr = GridSearchCV(mpr, param_space, n_jobs=-1, cv=cv_folds)
gs_mpr.fit(X_train, y_train)
```

```
print('Best parameters found:\n', gs_mpr.best_params_)
y_pred = gs_mpr.predict(X_val)
print("Best accuracy: ", rmse(y_val, y_pred))
print(classification_report(y_val, y_pred))
```

**Bayesian Optimization**

In [20]:
hidden_sz = X_train.shape[0] * 2 / 3 + 1
search_space = {
    "hidden_layer_sizes": Categorical([(hidden_layers,), (hidden_sz, hidden_sz * 2 / 3,), (hidden_sz, hidden_sz * 2 / 3, hidden_sz * 4 / 9)]),
    "activation": Categorical(["identity", "logistic", "tanh", "relu"]),
    "solver": Categorical("lbfgs", "adam", "sgd"),
    "alpha": Real(0.00001, 0.001)
    "batch_size": Categorical(['auto', n_samples // 5, n_samples // 10]),
    "learning_rate": Categorical(["constant", "invscaling", "adaptive"]),
    "learning_rate_init": Real(0.0005, 0.005),
    "max_iter": (200, 1000),
    "shuffle": Categorical([False, True]),
    "random_state": random_state,
    "tol": Real(0.0001, 0.1)
    "momentum": Real(0.75, 0.9),
    "nesterovs_momentum": Categorical([False, True]),
    "early_stopping": Categorical([False, True]),
    "validation_fraction": Real(0.1, 0.15),
    "beta_1": Real(0.75, 0.9),
    "beta_2": Real(0.85, 0.999),
    "epsilon": Real(1e-07, 1e-08),
    "n_iter_no_change": Integer(10, 15),
    "max_fun": Integer(10000, 15000),
}

In [ ]:
mpr = MLPRegressor()
bo_mpr = BayesSearchCV(svr, search_space, n_iter=20, n_jobs=-1, cv=cv_folds)
bo_mpr.fit(X_train, y_train)

In [ ]:
print('Best parameters found:\n', bo_mpr.best_params_)
y_pred = bo_mpr.predict(X_val)
print("Best accuracy: ", rmse(y_val, y_pred))
print(classification_report(y_val, y_pred))

### Feature Selection

In [31]:
mpr = MLPRegressor(**bo_mpr.best_params_)
selector = RFE(bo_mpr, step=1)
selector = selector.fit(X_train, y_train)
selector.ranking_

array([17,  1,  1,  1,  1,  8,  4,  1,  1,  1,  1,  1,  1, 12,  1,  6, 20,
       10, 22, 19,  1, 11,  1,  1, 25, 28,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  1, 18,  1,  3,
       21,  1,  1,  1,  1,  1,  1,  1,  1,  5,  1,  1,  1,  1,  1,  1,  1,
        9,  1,  1, 15, 14,  1,  1,  1, 13, 24,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  7,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1, 26, 30, 60, 53, 42, 57, 71, 68, 40, 43, 44, 46,
       48, 45, 54, 50, 58, 62, 63, 64, 67, 72, 66,  1,  1,  1, 23, 31, 74,
       56, 65, 41, 70, 69, 61, 73, 52, 59, 49, 55, 51, 38, 47, 36, 39, 33,
       34, 35, 37,  1,  1,  1,  1, 16, 27, 29, 32, 75, 76, 77, 78, 79, 80,
       81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92])

In [ ]:
mpr = MLPRegressor()
bo_mpr = BayesSearchCV(mpr, search_space, n_iter=50, n_jobs=-1, cv=cv_folds, random_state=random_state)
bo_mpr.fit(X_train.loc[:, selector.support_], y_train)

In [89]:
y_pred = bo_svr.predict(X_val.loc[:, selector.support_])

print('Best features found:\n', X_train.columns[selector.support_])
print("RMSE: ", rmse(y_val, y_pred))
print("MAE: ", mae(y_val, y_pred))

Best features found:
 Index(['LBP_BIN_bbox_13', 'LBP_BIN_bbox_14', 'LBP_BIN_bbox_15',
       'LBP_BIN_bbox_16', 'LBP_BIN_bbox_17', 'LBP_BIN_bbox_18',
       'LBP_BIN_bbox_19', 'LBP_BIN_bbox_20', 'LBP_BIN_bbox_21',
       'LBP_BIN_bbox_22', 'LBP_BIN_bbox_23', 'LBP_BIN_bbox_24',
       'LBP_BIN_bbox_25', 'SOBELX_BIN_bbox_0', 'SOBELX_BIN_bbox_1',
       'SOBELX_BIN_bbox_2', 'SOBELX_BIN_bbox_3', 'SOBELX_BIN_bbox_4',
       'SOBELX_BIN_bbox_5', 'SOBELX_BIN_bbox_6', 'SOBELX_BIN_bbox_7',
       'SOBELX_BIN_bbox_8', 'SOBELX_BIN_bbox_9', 'SOBELX_BIN_bbox_10',
       'SOBELX_BIN_bbox_11', 'SOBELX_BIN_bbox_12', 'SOBELX_BIN_bbox_13',
       'SOBELX_BIN_bbox_14', 'SOBELX_BIN_bbox_15', 'SOBELX_BIN_bbox_16',
       'SOBELX_BIN_bbox_17', 'SOBELX_BIN_bbox_18', 'SOBELX_BIN_bbox_19',
       'SOBELX_BIN_bbox_20', 'SOBELX_BIN_bbox_21', 'SOBELX_BIN_bbox_22',
       'SOBELX_BIN_bbox_23', 'SOBELX_BIN_bbox_24', 'SOBELX_BIN_bbox_25',
       'SOBELY_BIN_bbox_0', 'SOBELY_BIN_bbox_1', 'SOBELY_BIN_bbox_2',
       '

## Evaluation

#